In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


# This document is the attempt for the MCT gates
This attempt was abandomed due to the circuit depth inconsistencies and difficulty with different modes of the MCT function. This is also not adding new value to the project as these circuits had greater depth than those in the paper.

### The MCT gates
The gates indicated in the code comments in the circuit above can be replaced using the n-qubit Toffoli gate function MCT (multi-controlled-Toffoli gate). This function operates similarly to the Toffoli gate except that it can have more than 2 control qubits for a single target qubit. The function has 4 modes which are basic (2 ancilla), dirty basic ancilla (2 ancilla), advance (no ancilla), and no-ancilla. All modes are explored and the circuit depth and computing wall time examined.

This is the notation for the function:
mct(self, q_controls, q_target, q_ancilla, mode='basic')
self (QuantumCircuit) -- The QuantumCircuit object to apply the mct gate on.

q_controls (Union(QuantumRegister, list[Qubit])) -- The list of control qubits

q_target (Qubit) -- The target qubit

q_ancilla (Union(QuantumRegister, list[Qubit])) -- The list of ancillary qubits

mode (str) -- The implementation mode to use

In [ ]:
qcMCT = QuantumCircuit(7)
qcMCT.mct([0, 1, 2, 3], 4, [5, 6], mode = "v-chain-dirty")
qcMCT.draw()

In [ ]:
qcMCT.depth()

#### This is the BLANK mode

In [ ]:
## Coin and shift operator for walking on the number line with five-position states (Figure 4)

qc = QuantumCircuit(8, 5)
# label the qubits
q0 = 0
q1 = 1
q2 = 2
c0 = 3
q3 = 4
c1 = 5
q4 = 6
c = 7

# set the initial states |01111>
qc.x(q1)
qc.x(q2)
qc.x(q3)
qc.x(q4)
    
# the start of the actual circuit
qc.h(c)
qc.cx(c, q0)
qc.cx(c, q1)
qc.cx(c, q2)
qc.cx(c, q3)
qc.cx(c, q4)

# MCT gate replacement 
qc.mct([q0, q1, q2, q3], q4)
qc.mct([q0, q1, q2], q3)
#qc.mct([q0, q1, q2, q3], q4)

qc.ccx(q0, q1, c0)
qc.ccx(q0, q1, q2)
qc.cx(q0, q1)
qc.x(q0)
qc.cx(c, q0)
qc.cx(c, q1)
qc.cx(c, q2)
qc.cx(c, q3)
qc.cx(c, q4)

#qc.measure([q0, q1, q2, q3, q4], [0, 1, 2, 3, 4])

# diagram of one shift operator
qc.draw('mpl', initial_state = True)

In [ ]:
qc.depth()

In [ ]:
# apply the 13 steps used in the paper and measure the output
qc = QuantumCircuit(8, 5)
# label the qubits
q0 = 0
q1 = 1
q2 = 2
c0 = 3
q3 = 4
c1 = 5
q4 = 6
c = 7

# set the initial states |01111>
qc.x(q1)
qc.x(q2)
qc.x(q3)
qc.x(q4)
    
# the start of the actual circuit
for i in range(13):
    qc.h(c)
    qc.cx(c, q0)
    qc.cx(c, q1)
    qc.cx(c, q2)
    qc.cx(c, q3)
    qc.cx(c, q4)

# this part can be replaced by the MCT gate
    qc.mct([q0, q1, q2, q3], q4)
    qc.mct([q0, q1, q2], q3)
# end of the part that can be replaced by the MCT gate

    qc.ccx(q0, q1, c0)
    qc.ccx(q0, q1, q2)
    qc.cx(q0, q1)
    qc.x(q0)
    qc.cx(c, q0)
    qc.cx(c, q1)
    qc.cx(c, q2)
    qc.cx(c, q3)
    qc.cx(c, q4)

qc.measure([q0, q1, q2, q3, q4], [0, 1, 2, 3, 4])

In [ ]:
# get the start time
st = time.time()

simulator = Aer.get_backend('aer_simulator')

qc = transpile(qc, simulator)

# Run and get counts
# set measurement repition to 100 million
result = simulator.run(qc, shots = 100^6).result()
counts = result.get_counts(qc)

# get the end time
et = time.time()

plot_histogram(counts, title='Quantum Random Walk Counts')

In [ ]:
# apply the 13 steps used in the paper and measure the output
# this is a different mode of the MCT gate
qc = QuantumCircuit(8, 5)
# label the qubits
q0 = 0
q1 = 1
q2 = 2
c0 = 3
q3 = 4
c1 = 5
q4 = 6
c = 7

# set the initial states |01111>
qc.x(q1)
qc.x(q2)
qc.x(q3)
qc.x(q4)
    
# the start of the actual circuit
for i in range(13):
    qc.h(c)
    qc.cx(c, q0)
    qc.cx(c, q1)
    qc.cx(c, q2)
    qc.cx(c, q3)
    qc.cx(c, q4)

# this part can be replaced by the MCT gate
    qc.mct([q0, q1, q2, q3], q4, [c0, c1], "v-chain-dirty")
    qc.mct([q0, q1, q2], q3, [c0, c1], "v-chain-dirty")
# end of the part that can be replaced by the MCT gate

    qc.ccx(q0, q1, c0)
    qc.ccx(q0, q1, q2)
    qc.cx(q0, q1)
    qc.x(q0)
    qc.cx(c, q0)
    qc.cx(c, q1)
    qc.cx(c, q2)
    qc.cx(c, q3)
    qc.cx(c, q4)

qc.measure([q0, q1, q2, q3, q4], [0, 1, 2, 3, 4])

In [ ]:
# get the start time
st = time.time()

simulator = Aer.get_backend('aer_simulator')

qc = transpile(qc, simulator)

# Run and get counts
# set measurement repition to 100 million
result = simulator.run(qc, shots = 100^6).result()
counts = result.get_counts(qc)

# get the end time
et = time.time()

plot_histogram(counts, title='Quantum Random Walk Counts')

Leave this here for now because I can't figure out the circuit depth and different modes and these methods yield the same results and are not faster.